In [6]:
# Dans ce notebook :Constructions des data frames à partir de tous les xml dans le corpus 
# ces dataframes permettront d elaborer nos modèles de prédictions 

In [1]:
from xml.dom import minidom
xmldoc = minidom.parse('C:/Users/MyPC/Documents/AIC-partage/TC3/Projet/theatreClassique/XML/ABEILLE_ARGELIE.xml')

itemlist = xmldoc.getElementsByTagName('title')

In [2]:
x= itemlist[0]
print(len(x.childNodes))
y =x.childNodes[0]
y.nodeValue

1


'ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.'

In [2]:
import os
import xml.etree.ElementTree as ET 
from xml.dom import minidom

path = 'C:/Users/MyPC/Documents/AIC-partage/TC3/Projet/theatreClassique/XML/'
list_xml= []
for filename in os.listdir(path):
    
    if not filename.endswith('.xml'): continue
    try :
        fullname = os.path.join(path, filename)
        xmldoc = minidom.parse(fullname)
        list_xml.append(xmldoc)
    except  :
        print('{} is corrupt'.format(filename))
        
    

GOUGES_MOLIERENINON.xml is corrupt
GOULARD_AGIS.xml is corrupt


In [3]:
import nltk
import string, pprint, os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [4]:
def corpus( elements, indent ):
    c =[]
    
    for el in elements:
        #print("   " * indent + "nodeName: " + str(el.nodeName) )
        if str(el.nodeValue) != '\n\t\t\t' and str(el.nodeValue) !='None' and str(el.nodeValue) != None and '\n\t' not in str(el.nodeValue) :
            c.append((str(el.nodeValue)))
        #print("   " * indent + "childNodes: " + str(el.childNodes) )
        c.append(corpus(el.childNodes, indent + 1))
    return c

corpus1 = corpus( list_xml[0].getElementsByTagName('body'), 0 )

In [1]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def clean_corpus_bis(corpus_list):
    text = str (corpus_list)
    punct_killer = str.maketrans('', '', string.punctuation)
    text = text.translate(punct_killer)
    tokens = nltk.word_tokenize(text)
    tokens_clean = a = [t for t in tokens if t != '[' and t != ']' and t !=',' and t !='.']
    return tokens_clean, text

def clean_corpus(corpus_list):
    text = str (corpus_list)
    intab = "'" 
    outtab = " "
    trans = str.maketrans(intab, outtab)
    text = text.translate(trans)
    intab = "-" 
    outtab = " "
    trans = str.maketrans(intab, outtab)
    text = text.translate(trans)
    #    punct_killer = str.maketrans('', '', string.punctuation)
    # garder punctuation 
    punct_killer = str.maketrans('', '', '"#$%&\'()*+-/:<=>@[\\]^_`{|}~')
    text = text.translate(punct_killer)
    tokens = nltk.word_tokenize(text)
    #tokens_clean = a = [t for t in tokens if t != '[' and t != ']' and t !=',' and t !='.']

    tokens_clean = a = [t for t in tokens if t != '[' and t != ']' ]
    return tokens_clean, text




In [6]:
def titre(xml_doc):
    itemlist = xml_doc.getElementsByTagName('title')
    x= itemlist[0]
    y =x.childNodes[0]
    return y.nodeValue

In [7]:
def genre(xml_doc):   
    itemlist = xml_doc.getElementsByTagName('genre')
    x= itemlist[0]
    print(x)
    print(len(x.childNodes))
    
    print(x.childNodes)
    y =x.childNodes[0]
    
    y.nodeValue
    print
    


def location_histoire(xml_doc):
    itemlist = xml_doc.getElementsByTagName('set')
    x= itemlist
    dict= {}
    for el in x :
        d= {}
        location = el.getAttribute("location")
        periode = el.getAttribute("periode")
        country = el.getAttribute("country")
        dict["location histoire"]= location
        dict["periode histoire"]= periode
        dict["country histoire"]= country
        
       
    return dict 
    
def structure(xml_doc):
    itemlist = list_xml[0].getElementsByTagName('structure')
    x= itemlist[0]
    print(len(x.childNodes))
    y =x.childNodes[0]
    y.nodeValue
location_histoire(list_xml[0])

{'country histoire': 'Grèce',
 'location histoire': 'Larisse',
 'periode histoire': 'IIème avant JC'}

In [64]:
itemlist = list_xml[0].getElementsByTagName('SourceDesc')
x= itemlist[0]
print(len(x.childNodes))
y =x.childNodes[0]
y.nodeValue
#x.childNodestagName
x.childNodes
x.childNodes
for el in x.childNodes :
    if type(el)== minidom.Element:
        print(el._get_localName)
    if type(el)== minidom.Text:
        print(type(el))

15
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: genre at 0x1dbd508b768>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: inspiration at 0x1dbd508b800>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: structure at 0x1dbd508b898>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: type at 0x1dbd508b930>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: periode at 0x1dbd508b9c8>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: taille at 0x1dbd508ba60>>
<class 'xml.dom.minidom.Text'>
<bound method Element._get_localName of <DOM Element: permalien at 0x1dbd508baf8>>
<class 'xml.dom.minidom.Text'>


### Enregistre tous le xml sous forme de dataframe + découpage, mais pas utilisé au cours du projet

In [7]:
import pandas as pd 

#  minidom.Element.tagName , nom de balise
# minidom.Element.childNodes[0].nodeValue, noeud fils valeur à l interieur de la balise 
def info_text(list_xml, tag):
    #list_xml =list_xml[0:20]
    dict={}
    df = pd.DataFrame(data=dict)
    i= 0
    for doc in list_xml:
        i=i+1
        #print(i)
        itemlist = doc.getElementsByTagName(tag)
        x= itemlist[0]
        dict={}
        dict["Titre"]= titre(doc)
        lieux_dict = location_histoire(doc)
        for key, value in lieux_dict.items():
            dict[key]= value
        
        
        text =corpus( doc.getElementsByTagName('body'), 0 )
        
        text_clean = clean_corpus(text)[0]
        list_text_cut=[]
        l=[]
        for i in range( len(text_clean)):
            if i%1000==0 and i>0:
                list_text_cut.append(l)
                l=[]
            l.append(text_clean[i])
        
        for text in list_text_cut : 
            text_clean1 = str(nltk.Text(text))
                

            dict["Text"]= text_clean1

            for el in x.childNodes :
                if type(el)== minidom.Element:
                   # print(el.tagName)    # Nom de balise 
                    if(len(el.childNodes)>=1):
                        #print(el.childNodes[0].nodeValue)  # valeur de la balise correspondante
                        dict[el.tagName] = el.childNodes[0].nodeValue
            #print(dict)
            df_doc = pd.DataFrame(data=dict , index=[0])
            #print(df_doc)
            df = df.append(df_doc)

               # if type(el)== minidom.Text:
                    #print(el)
    return df


        
        
        
df =info_text(list_xml, 'SourceDesc') 
print(len(list_xml))


1025


# Fonction permettant d'enregistrer chaque texte + métadonnées (séparement) dans une dataframe

In [8]:
import pandas as pd 

#  minidom.Element.tagName , nom de balise
# minidom.Element.childNodes[0].nodeValue, noeud fils valeur à l interieur de la balise 
def info_text(list_xml, tag):
    #list_xml =list_xml[0:20]
    dict={}
    df = pd.DataFrame(data=dict)
    i= 0
    for doc in list_xml:
        i=i+1
        #print(i)
        itemlist = doc.getElementsByTagName(tag)
        x= itemlist[0]
        dict={}
        dict["Titre"]= titre(doc)
        lieux_dict = location_histoire(doc)
        for key, value in lieux_dict.items():
            dict[key]= value
        
        
        text =corpus( doc.getElementsByTagName('body'), 0 )
        text_clean = clean_corpus(text)[1]
        dict["Text"]= text_clean
        
        for el in x.childNodes :
            if type(el)== minidom.Element:
               # print(el.tagName)    # Nom de balise 
                if(len(el.childNodes)>=1):
                    #print(el.childNodes[0].nodeValue)  # valeur de la balise correspondante
                    dict[el.tagName] = el.childNodes[0].nodeValue
        #print(dict)
        df_doc = pd.DataFrame(data=dict , index=[0])
        #print(df_doc)
        df = df.append(df_doc)
        
           # if type(el)== minidom.Text:
                #print(el)
    return df


        
        
        
df =info_text(list_xml, 'SourceDesc') 
print(len(list_xml))


1025


In [9]:
# Enregistre base avec toutes les infos plus texte


df.to_csv("body_base_notcut_clean_punc.csv", sep='\t')



In [31]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

def handleToc(slides):
    for slide in slides:
        title = slide.getElementsByTagName("title")[0]
        print(title)
        print ("<p>%s</p>" % getText(title.childNodes))



In [29]:
import pandas as pd 
d = {'col1': [1, 2], 'col2': [3, 4]}
for  i in range(0,5):
    df = df.append(pd.DataFrame(data=d))
df

,col1,col2
0,6,18
1,12,24
0,1,3
1,2,4
0,1,3
1,2,4
0,1,3
1,2,4
0,1,3
1,2,4


In [1]:
#Creer nouvelle variable pour mapper celle existante
# Inculure directement la strucure dans le corpus 

import pandas as pd 
df = pd.read_csv('body_base_cut.csv',sep='\t',  encoding = "latin" )
df

,Unnamed: 0,Text,Titre,country histoire,dewey,encodage,genre,idBook,inspiration,keywords,...,note,periode,periode histoire,permalien,refDoc,refNum,sources,structure,taille,type
0,0,<Text: n ACTE I n SCÈNE I Argélie Clytie...>,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
1,0,<Text: faut prendre un Époux n n CLYTIE Votre...>,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
2,0,<Text: assembler Mais dans vos entretiens je n...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
3,0,<Text: Timagène Clytie n CLYTIE La Reine atten...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
4,0,<Text: Lois qui vous contraigne Et par mes seu...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
5,0,<Text: fait qui vous doive irriter Estce un cr...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
6,0,<Text: rigueur Me fitelle sentir sa nouvelle g...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
7,0,<Text: sensible à votre désespoir Obtiendra qu...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
8,0,<Text: rigoureux Je sais quaprès avoir désespé...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers
9,0,<Text: mes fers Publiez que je suis infidèle e...,"ARGÉLIE, REINE DE THESSALIE. TRAGÉDIE.",Grèce,NaN,NaN,Tragédie,NaN,histoire grecque,NaN,...,NaN,1671-1680,IIème avant JC,http://gallica.bnf.fr/ark:/12148/bpt6k5812122r,NaN,NaN,NaN,Cinq actes,1500-1750,vers


In [4]:
df_utile = df[['Text', 'Titre' ,'country histoire', 'periode', 'periode histoire','type', 'genre']]
del df

# Base de sortie (celle qu'on a utilisé )

In [5]:
df_utile = df_utile.sample(frac=1).reset_index(drop=True)
labels = list(df_utile.genre.unique())
df_utile['labels'] = df_utile['genre'].apply(labels.index)
df_utile


,Text,Titre,country histoire,periode,periode histoire,type,genre,labels
0,<Text: sapprête à faire le mal la fille de...>,"HERCULE FURIEUX, TRAGÉDIE.",Grèce,(Antique),Temps mythologiques,prose,Tragédie,0
1,<Text: n ACTE I n SCÈNE PREMIÈRE Jason Iphite...>,"MÉDÉE, TRAGÉDIE",Grèce,1691-1700,Temps mythologiques,vers,Tragédie,0
2,<Text: Seigneur que je déteste Mon coeur est t...,"OTHON, TRAGÉDIE",Italie,1661-1670,Ier avant JC,vers,Tragédie,0
3,<Text: cour quà régir lUnivers n n FABRICE Ah...>,"LE FANTOME AMOUREUX, TRAGI-COMÉDIE",Italie,1651-1660,XIIIème,vers,Tragi-comédie,1
4,<Text: part Lagréable figure n n JODELET à Don...,"LE JODELET OU LE MAÎTRE VALET, COMÉDIE.",Espagne,1641-1650,XVème,vers,Comédie,2
5,<Text: Comment n n n LISETTE n Oui laide...>,"LES SINCÈRES, COMÉDIE",France,1731-1740,XVIIIème,prose,Comédie,2
6,<Text: n THALIE Mon Oncle qui paraît va me...>,"LE CENTENAIRE DE MOLIÈRE, COMÉDIE.",France,1771-1780,XVIIème,mixte,Comédie,2
7,<Text: meilleure soeur qui ait jamais aimé une...,"BÉRÉNICE, TRAGI-COMÉDIE EN PROSE.",Crète,1631-1640,(inconnue),prose,Tragi-comédie,1
8,<Text: n n SCÈNE PREMIÈRE Pâris OEnone n PÂRIS...,"LE JUGEMENT DE PÂRIS, COMÉDIE.",Grèce,1771-1780,Temps mythologiques,prose,Comédie,2
9,<Text: bras sur mes épaules sur ma tête même...>,"HURLUBRELU OU TOUT DE TRAVERS, COMÉDIE",France,1791-1800,XVIIIème,prose,Comédie,2


### Rapide test de prédictions, sans modifier la base au préalable (peu important )

In [6]:
list_text = list(df_utile.Text)

In [7]:
list_label = list(df_utile['genre'].apply(labels.index))

In [8]:
test_size = int(len(list_text)/5)
test_set , train_set  = list_text[:test_size], list_text[test_size:]
test_label, train_label = list_label[:test_size], list_label[test_size:]


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_set )
X_train_counts.shape

(8536, 10036)

In [10]:
 #TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(8536, 10036)

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_label)

In [12]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(train_set, train_label)

In [13]:
import numpy as np

predicted = text_clf.predict(test_set)
np.mean(predicted == test_label)

0.54217432052483594

In [14]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train_set, train_label)
predicted_svm = text_clf_svm.predict(test_set)
np.mean(predicted == test_label)

D:\Programmes\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.54217432052483594

In [15]:
np.mean(predicted_svm == test_label)

0.51640112464854737

In [16]:
# Grid Search
# Here, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name (remember the arbitrary name we gave). 
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}


In [17]:
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train_set, train_label)

D:\Programmes\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [18]:

# To see the best mean score and the params, run the following code

gs_clf.best_score_
gs_clf.best_params_

{'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}

In [19]:
# Similarly doing grid search for SVM
from sklearn.model_selection import GridSearchCV
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(train_set, train_label)


gs_clf_svm.best_score_
gs_clf_svm.best_params_

D:\Programmes\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
D:\Programmes\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [20]:
predicted_svm_gs = gs_clf.predict(test_set)
np.mean(predicted_svm_gs == test_label)

0.5623242736644799

In [23]:

# Stemming Code

import nltk

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect), ('tfidf', TfidfTransformer()), 
                             ('mnb', MultinomialNB(fit_prior=False))])

text_mnb_stemmed = text_mnb_stemmed.fit(train_set, train_label)

predicted_mnb_stemmed = text_mnb_stemmed.predict(test_set)

np.mean(predicted_mnb_stemmed == test_label)

0.57169634489222121